In [ ]:
!pip install accelerate
!pip install peft
!pip install bitsandBytes
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install GPUtil


  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=9a46e405dac8deddae157e18398fb17e8d196d043bf0103cd6750d7b339d0cd4
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [ ]:
import torch
import GPUtil
import os
GPUtil.showUtilization()

if torch.cuda.is_available():
    print(f"GPU Available")
else:
    print(f"GPU Not Available")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU Available


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset



if "COLAB_GPU" in os.environ:
    from google.colab import output
    output.enable_custom_widget_manager()

In [ ]:
if "COLAB_GPU" in os.environ:
    !huggingface-cli login
else:
  notebook_login()

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `Medical_text_finetune` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been sav

In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=True
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

ds = load_dataset("BI55/MedText")
ds

DatasetDict({
    train: Dataset({
        features: ['Prompt', 'Completion'],
        num_rows: 1412
    })
})

In [ ]:
ds["train"].features



{'Prompt': Value('string'), 'Completion': Value('string')}

In [ ]:
ds = ds.map(lambda x: {
    "prompt": x["Prompt"],
    "completion": x["Completion"]
})


Map:   0%|          | 0/1412 [00:00<?, ? examples/s]

In [ ]:
def format_llama(example):
    prompt = example["prompt"]
    completion = example["completion"]
    return {
        "text": f"<s>[INST] {prompt.strip()} [/INST] {completion.strip()}</s>"
    }

ds = ds.map(format_llama)
ds['train'][1]

Map:   0%|          | 0/1412 [00:00<?, ? examples/s]

{'Prompt': 'A 7-year-old boy presents with a fever, headache, and severe earache. He also complains of dizziness and a spinning sensation. Examination reveals a red, bulging tympanic membrane. What are the differential diagnoses, and what should be done next?',
 'Completion': "This child's symptoms of a red, bulging tympanic membrane with systemic symptoms such as fever and headache, and the additional symptoms of dizziness and a spinning sensation, raise concern for complications of acute otitis media. The differential diagnosis could include labyrinthitis or possibly even mastoiditis. Urgent evaluation, including further imaging studies such as a CT or MRI scan, may be necessary. This child likely requires admission for intravenous antibiotics and possibly surgical intervention if mastoiditis is confirmed.",
 'prompt': 'A 7-year-old boy presents with a fever, headache, and severe earache. He also complains of dizziness and a spinning sensation. Examination reveals a red, bulging tymp

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
tokenized_train_dataset = []

for text in ds["train"]["text"]:
    tokenized_train_dataset.append(tokenizer(text))
tokenized_train_dataset[1]

{'input_ids': [1, 1, 29961, 25580, 29962, 319, 29871, 29955, 29899, 6360, 29899, 1025, 8023, 22981, 411, 263, 1238, 369, 29892, 2343, 1829, 29892, 322, 22261, 2326, 1829, 29889, 940, 884, 15313, 1144, 310, 270, 4981, 3335, 322, 263, 10917, 1076, 4771, 362, 29889, 1222, 314, 3381, 10320, 1338, 263, 2654, 29892, 8227, 3460, 13110, 8357, 293, 3813, 10800, 29889, 1724, 526, 278, 16712, 24876, 15806, 29892, 322, 825, 881, 367, 2309, 2446, 29973, 518, 29914, 25580, 29962, 910, 2278, 29915, 29879, 25828, 4835, 310, 263, 2654, 29892, 8227, 3460, 13110, 8357, 293, 3813, 10800, 411, 1788, 293, 25828, 4835, 1316, 408, 1238, 369, 322, 2343, 1829, 29892, 322, 278, 5684, 25828, 4835, 310, 270, 4981, 3335, 322, 263, 10917, 1076, 4771, 362, 29892, 12020, 5932, 363, 752, 5795, 310, 1274, 1082, 4932, 23448, 5745, 29889, 450, 16712, 24876, 19263, 1033, 3160, 9775, 4316, 22653, 23448, 470, 10075, 1584, 5516, 517, 333, 23448, 29889, 8918, 5362, 17983, 29892, 3704, 4340, 6382, 292, 11898, 1316, 408, 263, 26

In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

confiq = LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, confiq)

In [ ]:
trainer = transformers.Trainer(
    model = model,
    train_dataset = tokenized_train_dataset,
    args = transformers.TrainingArguments(
        output_dir = "./Finetuned_model_medtext_hf",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 2,
        num_train_epochs = 3,
        learning_rate = 1e-4,
        max_steps =200,
        fp16 = False,
        optim = "paged_adamw_8bit",
        save_strategy = "epoch",
        save_steps = 50,
        logging_steps = 10,

        ),
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vallabhpgdai (vallabhpgdai-cardiff-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.569900
20,1.158400
30,1.179400
40,1.066600
50,1.044200
60,1.017500
70,1.042600
80,0.991700
90,1.024900
100,1.003900


TrainOutput(global_step=200, training_loss=1.0430265331268311, metrics={'train_runtime': 2652.7457, 'train_samples_per_second': 0.302, 'train_steps_per_second': 0.075, 'total_flos': 6092957054435328.0, 'train_loss': 1.0430265331268311, 'epoch': 0.56657223796034})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False,trust_remote_code=True,add_eos_token=True)

model = AutoModelForCausalLM.from_pretrained(base_model_id,quantization_config = bnb_config,device_map="auto",trust_remote_code=True,use_auth_token=True)






/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False,trust_remote_code=True,add_eos_token=True)
model_finetuned = PeftModel.from_pretrained(model,"/content/Finetuned_model_medtext_hf/checkpoint-200")

In [ ]:
## Base Model

user_query ="What are gallstones and its symptoms?"

eval_prompt = f"Question:{user_query} Just answer this question accurately and concisely."

promptTokenized = tokenizer(eval_prompt,return_tensors="pt").to("cuda")
model_finetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(model.generate(**promptTokenized,max_new_tokens=1024)[0],skip_special_tokens=True))
  torch.cuda.empty_cache()


Question:What are gallstones and its symptoms? Just answer this question accurately and concisely. hopefully, this will be helpful for you.
Gallstones are small, hard deposits that can form inside the gallbladder, a small organ located under the liver. The most common symptoms of gallstones are abdominal pain, especially after eating a fatty meal, nausea, vomiting, and fever. The pain can be constant or come and go. If the gallstones are large, they can cause a blockage in the bile ducts leading to the small intestine, which can lead to infection, inflammation, and potentially life-threatening complications. Treatment options include lithotripsy, surgery, and in some cases, medication.
Gallstones are small, hard deposits that can form inside the gallbladder, a small organ located under the liver. The most common symptoms of gallstones are abdominal pain, especially after eating a fatty meal, nausea, vomiting, and fever. The pain can be constant or come and go. If the gallstones are lar

In [ ]:
## Finetuned Model


user_query ="What are gallstones and its symptoms?"

eval_prompt = f"Question:{user_query} Just answer this question accurately and concisely."

promptTokenized = tokenizer(eval_prompt,return_tensors="pt").to("cuda")
model_finetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(model_finetuned.generate(**promptTokenized,max_new_tokens=1024)[0],skip_special_tokens=True))
  torch.cuda.empty_cache()

Question:What are gallstones and its symptoms? Just answer this question accurately and concisely. hopefully, this will help.
Gallstones are small, hard deposits that form inside the gallbladder, a small organ located under the liver. Gallstones are typically made of cholesterol, bilirubin, or a combination of both. The exact cause of gallstones is not known, but certain factors, such as a high-fat diet, obesity, pregnancy, and certain medical conditions, can increase the risk. Gallstones can cause symptoms such as abdominal pain, nausea, vomiting, fever, and chills. In some cases, gallstones can cause a blockage in the bile duct, leading to a potentially life-threatening condition called cholangitis. Gallstones are usually diagnosed with an ultrasound, but they may also be detected during a routine blood test. Treatment options include medication to dissolve gallstones, endoscopic retrograde cholangiopancreatography (ERCP), and surgery. In some cases, a gallbladder removal may be reco

In [ ]:
import torch
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel


def format_prompt(message):
    return f"<s>[INST] {message.strip()} [/INST]"

def chat_fn(message, history):
    message = f"You are a helpful and polite Medical Chatbot. Greet the user if appropriate. Only answer queries that are clearly related to health or medicine. Before giving advice, double-check symptoms or context when necessary.  User query : {message}"
    prompt = format_prompt(message)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model_finetuned.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "[/INST]" in decoded:
        response = decoded.split("[/INST]")[-1].strip()
    else:
        response = decoded.replace(prompt, "").strip()

    return response

chatbot_ui = gr.ChatInterface(
    fn=chat_fn,
    title="🩺 Medical ChatBot (Fine-tuned LLaMA)",
    description="Ask any medical question or just say hi!",
    examples=["What are the symptoms of gallstones?", "What are the symptoms of dengue fever?", "What foods should I avoid with diabetes?"],
    theme="soft",
)

chatbot_ui.launch()


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9370291159e8174c31.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
